In [ ]:
import model
import tf_keras as keras
import pandas as pd
import tensorflow as tf
import crystal_loader
from tqdm import tqdm
import tqdm.keras

from symmetry import *
import dill
from sklearn.preprocessing import StandardScaler
import h5py
from sklearn.model_selection import train_test_split


In [ ]:
pickles_path = "../pickles/"
DNAME = "TiO2_2015"

features, labels = crystal_loader.load_dset(pickles_path, DNAME)

labels = np.array(labels)

def scale_ragged(features):
    stacked = np.vstack(features)
    SSC = StandardScaler().fit(stacked)
    scaled_features = [SSC.transform(struct) for struct in features]

    return scaled_features

scaled_features = scale_ragged(features)

Xtrain, Xtest, y_train, y_test = train_test_split(scaled_features, labels, shuffle=True, random_state=12, test_size=0.4)
Xval, Xtest, y_val, y_test = train_test_split(Xtest, y_test, shuffle=True, random_state=12, test_size=0.5)

Xtrain = tf.ragged.constant(Xtrain, ragged_rank=1, inner_shape=(70,))
Xval = tf.ragged.constant(Xval, ragged_rank=1, inner_shape=(70,))
Xtest = tf.ragged.constant(Xtest, ragged_rank=1, inner_shape=(70,))

In [ ]:
from importlib import reload
model = reload(model)
MLPNet = model.MLPNet

In [ ]:
layers = [keras.layers.Dense(50, activation="relu"), keras.layers.Dense(20, activation="relu")] + list([keras.layers.Dense(10, activation="relu") for i in range(10)])

MLP1 = MLPNet(layers=layers, N_features=70)

MLP1.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.0003),
    loss = keras.losses.MeanSquaredError(),
    metrics = [keras.metrics.RootMeanSquaredError()]
)


In [ ]:
print(Xtrain.shape)

In [ ]:
res = MLP1.fit(
    Xtrain, y_train,
    batch_size = 50,
    epochs = 5,
    # validation_data = (Xval, yval),
    verbose = 0,
    validation_data=(Xval, y_val),
    callbacks=[]
)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot(res.epoch, res.history["loss"], label="training")
ax.plot(res.epoch, res.history["val_loss"], label="validation")

ax.legend()